<a href="https://colab.research.google.com/github/aaleksandraristic/GA-s-Stock-Prediction-/blob/main/GAs_Project_Shapelet_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tslearn

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.shapelets import ShapeletModel
from tslearn.shapelets import grabocka_params_to_shapelet_size_dict
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
data = pd.read_csv('/content/final_dataset.csv')

# Assume 'Date' is a datetime column
data['Date'] = pd.to_datetime(data['Date'])

# Sort the data by date
data.sort_values('Date', inplace=True)

# Use only 'Date' and 'IBM' columns
data = data[['Date', 'INTC']]

# Set 'Date' as the index
data.set_index('Date', inplace=True)

# Feature engineering: Creating a lag feature
data['INTC_Lag1'] = data['INTC'].shift(1)

# Drop missing values
data.dropna(inplace=True)

In [ ]:
# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train, test = data[0:train_size], data[train_size:]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# Create time series datasets
X_train, y_train = [], []
X_test, y_test = [], []

# Use both 'IBM' and 'IBM_Lag1' as features
for i in range(1, len(train_scaled)):
    X_train.append(train_scaled[i - 1:i])
    y_train.append(train_scaled[i, 0])

for i in range(1, len(test_scaled)):
    X_test.append(test_scaled[i - 1:i])
    y_test.append(test_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)


In [ ]:
# Discretize the target variable for classification
n_classes = 3  # You can adjust the number of classes based on your preference
y_train_discrete = np.digitize(y_train, np.linspace(0, 1, n_classes + 1)[1:]) - 1
y_test_discrete = np.digitize(y_test, np.linspace(0, 1, n_classes + 1)[1:]) - 1

# Build a shapelet model
shapelet_sizes = grabocka_params_to_shapelet_size_dict(n_ts=X_train.shape[0],
                                                       ts_sz=X_train.shape[1],
                                                       n_classes=len(set(y_train_discrete)),
                                                       l=1,
                                                       r=30)

shp_clf = ShapeletModel(n_shapelets_per_size=shapelet_sizes, optimizer="sgd", weight_regularizer=.01,
                        max_iter=300)

/usr/local/lib/python3.10/dist-packages/tslearn/shapelets/shapelets.py:354: FutureWarning:

The default value for 'scale' is set to False in version 0.4 to ensure backward compatibility, but is likely to change in a future version.



In [ ]:
# Fit the model
shp_clf.fit(X_train, y_train_discrete, )

# Predictions
y_pred = shp_clf.predict(X_test)

# Inverse transform the predicted classes to get continuous predictions
# Here, I'm using the mean value of each bin as the predicted value
# Get bin edges
bin_edges = np.linspace(0, 1, n_classes + 1)

# Calculate midpoints of the bins
bin_midpoints = (bin_edges[1:] + bin_edges[:-1]) / 2

# Inverse transform the predicted classes using bin midpoints
y_pred_continuous = bin_midpoints[y_pred]


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Number of future points to forecast
future_steps = 1000  # You can adjust this based on your needs

# Use the last data point from the test set to initialize the prediction
last_point = test_scaled[-1]

# Initialize lists to store future predictions and directions
future_predictions = []
directions = []

# Generate future predictions and directions
for _ in range(future_steps):
    # Reshape the last_point to match the input shape of the model
    input_data = last_point.reshape((1, 1, last_point.shape[0]))

    # Make a prediction for the next time step
    next_step = shp_clf.predict(input_data)

    # Inverse transform the predicted class to get the continuous prediction
    next_step_continuous = bin_midpoints[next_step][0]

    # Append the prediction to the list
    future_predictions.append(next_step_continuous)

    # Determine the direction (increase or decrease)
    direction = 1 if next_step_continuous > last_point[-1] else -1
    directions.append(direction)

    # Update last_point for the next iteration
    last_point = np.vstack((last_point[1:], np.array([next_step_continuous])))

# Print or use future_predictions and directions as needed
print("Future Predictions:", future_predictions)


# Generate future timestamps for the predictions
future_timestamps = pd.date_range(start=test.index[-1] + pd.DateOffset(1), periods=future_steps, freq='B')

# Plot the original data and the predicted future direction as a bar plot
fig = go.Figure()

# Plot the original data
fig.add_trace(go.Scatter(x=test.index, y=test['INTC'], mode='lines', name='Original Data'))

# Create a bar plot for the predicted future values with different colors for increase and decrease
for timestamp, prediction, direction in zip(future_timestamps, future_predictions, directions):
    color = 'green' if direction == 1 else 'red'
    fig.add_trace(go.Bar(x=[timestamp], y=[prediction], marker_color=color, name='Predictions'))

# Add layout information
fig.update_layout(title='Time Series Forecasting with Shapelet Model',
                  xaxis_title='Timestamp',
                  yaxis_title='Value')

# Show the plot
fig.show()


Future Predictions: [0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.16666666666666666, 0.16666666666666666, 0.8333333333333333, 0.166666

In [ ]:
from sklearn.metrics import mean_squared_error

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred_continuous)

# Print the MSE
print("Mean Squared Error:", mse)


Mean Squared Error: 0.15755270511448563
